# Ejemplo de Webscraping

En este notebook sencillo generaremos un archivo .txt fruto de obtener todo el contenido en texto plano de una web mencionada a través de la librerías requests y BeautifulSoup. Usaremos este .txt para hacer análisis de wordcount.

El código contiene el seguiente detalle:

* Importaciones de módulos: módulos requests (para realizar solicitudes HTTP), BeautifulSoup (para analizar HTML) y funciones específicas de urllib para manipular URLs.

* Función scrape_url: acepta varios parámetros:

  * url: La URL que se va a rascar.
  * max_levels: La profundidad máxima a la que se realizará el web scraping.
  * current_level: El nivel actual de profundidad (por defecto es 1).
  * visited: Un conjunto para mantener un registro de las URLs visitadas (por defecto es None).
  * output_file: El nombre del archivo de salida donde se guarda el texto extraído (por defecto es "output.txt").

* Verificación del nivel actual y profundidad máxima: Si current_level supera max_levels, la función se detiene para evitar un scraping demasiado profundo.

* Verificación de URLs visitadas: Si la URL actual ya está en el conjunto visited, la función también se detiene para evitar ciclos infinitos en el scraping.

* Solicitud HTTP y análisis de HTML: Se realiza una solicitud HTTP a la URL utilizando requests.get(). Si la respuesta es exitosa (código de estado 200), se analiza el HTML de la página utilizando BeautifulSoup.

* Extracción de texto plano y escritura en el archivo de salida: Se extrae el texto plano del HTML utilizando soup.get_text(), se agrega un salto de línea adicional para separar los contenidos de las páginas, y se guarda en un archivo de salida especificado por output_file.

* Recursión para seguir enlaces internos: Se encuentran todos los enlaces en la página y se itera sobre ellos. Se construye la URL absoluta utilizando urljoin(). Si el enlace pertenece al mismo esquema y dominio que la URL original, se llama recursivamente a la función scrape_url para seguir scrapeando esa URL en el siguiente nivel de profundidad.

* Manejo de excepciones: Cualquier error que ocurra durante el proceso de scraping se imprime en la consola.

In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def scrape_url(url, max_levels, current_level=1, visited=None, output_file="output.txt"):
    if visited is None:
        visited = set() #Declare a set in order to insert visited websites. 

    if current_level > max_levels: # This declares the limit of the number of levels you wish to explore within a site. 
        return

    if url in visited: # If the url has already been visited we are going over duplicate values. 
        return

    print("Scraping:", url) # Una vez ingresemos en una pagina web, la vamos a añadir a nuestro set. 
    visited.add(url)

    try:
        response = requests.get(url) #Intentamos meternos en la pagina web
        if response.status_code == 200: #En caso de que funcione la conexion, parseamos el HTML, para poder extraer el texto de la pagina sin la estructura. 
            soup = BeautifulSoup(response.text, 'html.parser') #Extraemos todo el texto de cada pagina web por su parte. 

            # Extraer texto plano de la página y guardarlo en el archivo de salida
            text = soup.get_text(separator="\n", strip=True) # Quitamos los espacios y separamos por frases. 
            #Función para guardar el contenido de cada pagina web en nuestro archivo de salida. 
            with open(output_file, "a", encoding="utf-8") as file:
                file.write(text + "\n\n")

            # Encuentra todos los enlaces en la página y sigue scraping si son parte de la misma ruta
            base_url = urlparse(url) 
            # ? - Una vez hayamos escrito el texto de la página web, debemos de coger el url, y esta función de urlparse
            # ? - Nos divide el url en varios componentes. 
            for link in soup.find_all('a', href=True): 
                # * - Extract all links found on the website with tags 'a'
                next_url = urljoin(url, link['href']) #Get the next url 
                parsed_next_url = urlparse(next_url) # Parse it to extract the structure
                # Here we're verifying that they share the same scheme and netloc of the url's so we don't dive into external sites. 
                if parsed_next_url.scheme == base_url.scheme and parsed_next_url.netloc == base_url.netloc:
                    scrape_url(next_url, max_levels, current_level=current_level + 1, visited=visited, output_file=output_file) #Scrape the next url 

    except Exception as e:
        print(f"Error scraping {url}: {e}")

Example

* Scheme: https
* Netloc: www.example.com
* Path: /path/page.html
* Query: id=10&category=books
* Fragment: section1

In [5]:
# Ejemplo de uso:
starting_url = "https://www.iana.org/help/example-domains"
max_levels = 4
output_file = "output.txt"

scrape_url(starting_url, max_levels, output_file=output_file)

Scraping: https://www.iana.org/help/example-domains
Scraping: https://www.iana.org/
Scraping: https://www.iana.org/about/
Scraping: https://www.iana.org/domains
Scraping: https://www.iana.org/protocols
Scraping: https://www.iana.org/numbers
Scraping: https://www.iana.org/about
Scraping: https://www.iana.org/domains/
Scraping: https://www.iana.org/numbers/
Scraping: https://www.iana.org/protocols/
Scraping: https://www.iana.org/performance
Scraping: https://www.iana.org/procedures
Scraping: https://www.iana.org/about/presentations
Scraping: https://www.iana.org/reports
Scraping: https://www.iana.org/about/framework
Scraping: https://www.iana.org/reviews
Scraping: https://www.iana.org/about/audits
Scraping: https://www.iana.org/about/excellence
Scraping: https://www.iana.org/contact
Scraping: https://www.iana.org/domains/root
Scraping: https://www.iana.org/domains/int
Scraping: https://www.iana.org/domains/arpa
Scraping: https://www.iana.org/domains/idn-tables
Scraping: https://www.iana.

Nota: este tipo de métodos, realizan numerosas solicitudes sobre los servidores webs de la página que estamos consultando. Es probable que el servidor acabe baneando nuestra conexión salvo que se enmascare.